In [ ]:
import numpy as np
from matplotlib import pyplot as plt
dt = 0.1  # Time step
T = 10  # Total time
omega = 1.0  # Angular velocity (rad/s)
acc_z = 1.0  # Linear acceleration along z-axis (m/s^2)

# Time array
time = np.arange(0, T + dt, dt)

# Delta orientation (rotation vector in IMU frame)
delta_orientation = np.zeros((len(time), 3))
delta_orientation[:, 2] = omega * dt  # Incremental yaw rotation around z-axis

# Filtered acceleration in the IMU frame
filtered_acceleration = np.zeros((len(time), 3))
filtered_acceleration[:, 2] = acc_z + 9.81

# Initialize variables
t_prev = time[0]
vel_prev = np.zeros(3)  # Initial velocity (assume zero)
position = [np.zeros(3)]  # Initial position (at origin)
total_rotation = np.eye(3)  # Initial orientation (identity matrix)

# Gravity in world frame
gravity_world = np.array([0, 0, -9.81])  # Downward acceleration due to gravity

In [ ]:
def rotation_matrix(roll, pitch, yaw):
    """
    """
    c_yaw, s_yaw = np.cos(yaw), np.sin(yaw)  # cos(α), sin(α)
    c_pitch, s_pitch = np.cos(pitch), np.sin(pitch)  # cos(β), sin(β)
    c_roll, s_roll = np.cos(roll), np.sin(roll)  # cos(γ), sin(γ)
    
    # Rotation matrix computation
    R = np.array([
        [c_yaw * c_pitch, c_yaw * s_pitch * s_roll - s_yaw * c_roll, c_yaw * s_pitch * c_roll + s_yaw * s_roll],
        [s_yaw * c_pitch, s_yaw * s_pitch * s_roll + c_yaw * c_roll, s_yaw * s_pitch * c_roll - c_yaw * s_roll],
        [-s_pitch, c_pitch * s_roll, c_pitch * c_roll]
    ])
    
    return R

In [ ]:
# Loop through data
for t, d_orient, ac in zip(time[1:], delta_orientation[1:], filtered_acceleration[1:]):
    dt = t - t_prev
    delta_rot = rotation_matrix(*d_orient)  
    total_rotation = delta_rot @ total_rotation 
    acc_world = total_rotation @ ac - gravity_world
    vel = vel_prev + acc_world * dt
    new_pos = position[-1] + vel * dt
    position.append(new_pos)
    vel_prev = vel
    t_prev = t

position = np.array(position)

fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')
# Plot the trajectory
ax.plot(position[:,0], position[:,1], position[:,2], label="Trajectory")